<a href="https://colab.research.google.com/github/nboravlev/parsing/blob/main/yandex_geocoder.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests
import pandas as pd

# Ваш API-ключ Яндекс-геокодера
API_KEY = "9dc56f2b-110b-4aba-a0ad-2d6dc07cf70e"

# Файл Excel с адресами для геокодирования
excel_file = "source.xlsx"

# Чтение адресов из Excel-файла в pandas DataFrame
df = pd.read_excel(excel_file, sheet_name="График")

# Создание списка адресов для геокодирования
addresses = df["Адрес объекта исходный"].tolist()

# Список для хранения результатов геокодирования
results = []

# Отправка запроса к Яндекс-геокодеру для каждого адреса
for address in addresses:
    url = f"https://geocode-maps.yandex.ru/1.x/?apikey={API_KEY}&geocode={address}&format=json&results=1"
    response = requests.get(url)
    if response.status_code == 200:
        # Извлечение координат из ответа Яндекс-геокодера
        json_data = response.json()
        pos = json_data["response"]["GeoObjectCollection"]["featureMember"][0]["GeoObject"]["Point"]["pos"]
        lon, lat = map(float, pos.split())
        results.append((address, lat, lon))
    else:
        print(f"Ошибка при геокодировании адреса {address}: {response.status_code}")

# Сохранение результатов в новый Excel-файл
result_df = pd.DataFrame(results, columns=["Адрес", "Широта", "Долгота"])
result_df.to_excel("результаты.xlsx", index=False)


In [ ]:
!pip install geopandas

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 11.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 48.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 75.6 MB/s eta 0:00:00


In [ ]:
import geopandas as gpd
import pandas as pd

In [ ]:

# Чтение данных из Excel-файла
df = pd.read_excel("результаты.xlsx")

# Создание GeoDataFrame на основе данных из DataFrame
gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df["Долгота"], df["Широта"]), crs="EPSG:4326")

# Экспорт GeoDataFrame в GeoJSON файл
gdf.to_file("результаты.geojson", driver="GeoJSON")

In [ ]:
import folium
import geopandas as gpd

# Чтение данных из GeoJSON файла
gdf = gpd.read_file("результаты.geojson")

# Создание интерактивной карты с помощью библиотеки folium
m = folium.Map(location=[gdf["Широта"].mean(), gdf["Долгота"].mean()], zoom_start=10)

# Добавление объектов на карту
for idx, row in gdf.iterrows():
    folium.Marker([row["Широта"], row["Долгота"]], tooltip=row["Адрес"]).add_to(m)

# Отображение карты
m
